
<section style="color: #fff;
    text-align: center;
    background-color: #337ab7;
    background-image: linear-gradient(120deg, #155799, #337ab7, #155799);
    padding: 1.5rem 2rem;
    margin: 0px 0 20px;
    border-bottom: 1px solid #eee;">
  <h1 class="title toc-ignore project-name" style="background-color:transparent;">Les principes FAIR appliqués à la bioinformatique</h1>
  <h2 class="subtitle project-tagline">Outils et environnements FAIR pour la bioanalyse</h3>
  <h3 class="subtitle project-tagline">Thomas Denecker - <i>Data brokering</i> - IFB core</h4> 
  
<div id="banner">
    <div style="display: inline-block;">
        <a href="https://twitter.com/DeneckerThomas" target="_blank"> <img src='https://upload.wikimedia.org/wikipedia/fr/c/c8/Twitter_Bird.svg' width='30px' style='margin: 10px 10px 0px 10px;'> </a>
    </div>
    <div style="display: inline-block;">
        <a href="mailto:thomas.denecker@france-bioinformatique.fr" target="_blank"> <img src='https://upload.wikimedia.org/wikipedia/fr/a/a7/Mail_%28Apple%29_logo.png' width='30px' style='margin: 10px 10px 0px 10px;'> </a>
    </div>
    <div style="display: inline-block;">
        <a href="https://github.com/thomasdenecker" target="_blank"> <img src='https://github.githubassets.com/images/modules/logos_page/Octocat.png' width='30px' style='margin: 10px 10px 0px 10px;' > </a>
    </div>
</div>
</section>

## Rejouer son article en un click

### Cas d'étude 

#### Contexte
L'objectif de l'étude est d'étudier la réponse à une privation en fer chez l'algue verte *Ostreococcus tauri* [Lelandais et al, 2016](https://doi.org/10.1186/s12864-016-2666-6). Il s'agit d'un organisme de 13.0328 Mb séparé en 20 chromosomes.

L'expérience est composée de 16 échantillons de données RNAseq (triplicat, single-end de 100bp). Le plan d'expérience est illustré ci-dessous. Pour accompagner cette démonstration, nous avons choisi les échantillons S11 et S12 (reponse adaptative à long terme,)

![Figure 1](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/planExperimental.png)

**Figure 1** | Plan expérimental 

#### Données utilisées
- Sequence: [GCF_000214015.3_version_140606_genomic.fna](https://www.ncbi.nlm.nih.gov/assembly/GCF_000214015.3/)
- Annotation: GCF_000214015.3_version_140606_genomic.gff
- Données RNAseq : des [données réuduites](https://doi.org/10.5281/zenodo.3997237) au chromosome 18 (pour la démo). Le projet avec les données non réduites est disponible sur [SRA](https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR4026187).

#### Analyse de données 

Le but de l'analyse de données est de mettre en évidence une liste de gènes différentiellement exprimés lorsque que le milieu de culture à une concetration en fer plus faible que dans un milieu standard. Les différentes étapes sont présentées dans la figure suivante : 

![Figure 2](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/DataAnalysis.png)

**Figure 2** | Étape de l'analyse de données 

## Comment rejouer toute l'analyse en un click ? 

Un reviewer souhaite refaire vos analyses ? Vous souhaitez analyser de nouvelles données exactement de la même façon que précédement ? 

Si c'est FAIR, c'est sans soucis ! Vous n'êtes pas convaincu ? Une démo ? 

<img style="margin: 0px auto;" src="https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/connexion.jpg" width="400"> 

### Étape 1 - Identification à Jupyter Hub

![Figure 3](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_1.png)

### Étape 2 - Création d'un job Jupyter Lab

![Figure 4](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_2.png)

### Étape 3 - Création / ouverture d'un notebook Jupyter

![Figure 5](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_3.png)

### Étape 4 - Import du code et des données 

![Figure 5](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_4.png)

#### Création d'un répertoire de travail

In [ ]:
mkdir demo
cd demo 

#### Import des données

**Téléchargement des données**

In [ ]:
wget https://zenodo.org/record/3997237/files/FAIR_Bioinfo_data.tar.gz?download=1 -O FAIR_Bioinfo_data.tar.gz

**Extraction des données**

In [ ]:
tar -xvzf FAIR_Bioinfo_data.tar.gz

#### Récupération des scripts d'analyse

**Snakemake files - Téléchargement**

In [ ]:
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/config.yml
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/demo.smk

**Snakemake files - Contenu**

``` Python
SAMPLES, = glob_wildcards(config["dataDir"]+"{sample}.fastq.gz")
BIDX = ["1","2","3","4","rev.1","rev.2"]

rule all:
  input:
    expand("FastQC/{sample}_fastqc.html", sample=SAMPLES),
    expand("Tmp/Otauri.{ext}.bt2", ext=BIDX),
    expand("Tmp/{sample}.sam", sample=SAMPLES),
    expand("Result/{sample}_sort.bam.bai", sample=SAMPLES),
    expand("Result/Counts/{sample}_ftc.txt", sample=SAMPLES),
    "Result/counts_matrix.txt"

rule matrix_counts:
  output:
    "Result/counts_matrix.txt"
  input:
    countfile=expand("Tmp/{sample}_ftc7.txt", sample=SAMPLES),
    geneID=expand("Tmp/{sample}_ftc1.txt", sample=SAMPLES)
  log:
    "Logs/matrix_counts.log"
  shell:
    """cp {input.geneID[0]} Tmp/ftc_geneID.txt ; paste Tmp/ftc_geneID.txt {input.countfile} > {output}"""

rule extract_counts:
  output:
    col7="Tmp/{sample}_ftc7.txt",
    col1="Tmp/{sample}_ftc1.txt"
  input:
    "Result/Counts/{sample}_ftc.txt"
  shell: """cut -f 7 {input} | sed 1d > {output.col7} ; cut -f 1 {input} | sed 1d > {output.col1} """

rule counting:
  output:
    "Result/Counts/{sample}_ftc.txt"
  input:
    bam="Result/{sample}_sort.bam",
    annot=config["dataDir"]+config["annots"]
  params: t="gene", g="ID", s="2"
  log:
    "Logs/{sample}_counts.log"
  conda: "condaEnv4SmkRules/counting.yml"
  shell:
    "featureCounts -t {params.t} -g {params.g} -a {input.annot} -s {params.s} -o {output} {input.bam} &> {log}"

rule sam2bam_sort:
  output:
    bam="Result/{sample}_sort.bam",
    bai="Result/{sample}_sort.bam.bai"
  input:
    "Tmp/{sample}.sam"
  log:
    sort="Logs/{sample}_sam2bam_sort.log",
    index="Logs/{sample}_bam2bai.log"
  conda: "condaEnv4SmkRules/sam2bam_sort.yml"
  shell: 
    "samtools sort -O bam -o {output.bam} {input} 2> {log.sort} ;"
    "samtools index {output.bam} 2> {log.index}"


rule bwt2_mapping:
  output:
    "Tmp/{sample}.sam"
  input:
    config["dataDir"]+"{sample}.fastq.gz",
    expand("Tmp/Otauri.{ext}.bt2", ext=BIDX)
  log:
    "Logs/{sample}_bwt2_mapping.log"
  conda: "condaEnv4SmkRules/bwt2_mapping.yml"
  shell: "bowtie2 -x Tmp/Otauri -U {input[0]} -S {output} 2> {log} "

rule genome_bwt2_index:
  output:
    expand("Tmp/Otauri.{ext}.bt2", ext=BIDX)
  input:
    config["dataDir"]+config["genome"]
  log:
    log1="Logs/genome_bwt2_index.log1",
    log2="Logs/genome_bwt2_index.log2"
  conda: "condaEnv4SmkRules/bwt2_mapping.yml"
  shell: "bowtie2-build {input} Tmp/Otauri 1>{log.log1} 2>{log.log2}"

rule fastqc:
  output: 
    "FastQC/{sample}_fastqc.zip",
    "FastQC/{sample}_fastqc.html"
  input: 
    config["dataDir"]+"{sample}.fastq.gz"
  log:
    log1="Logs/{sample}_fastqc.log1",
    log2="Logs/{sample}_fastqc.log2"
  conda: "condaEnv4SmkRules/fastqc.yml"
  shell: "fastqc --outdir FastQC/ {input} 1>{log.log1} 2>{log.log2}"
```

**Contenu de config.yml**

``` YAML
genome:
  O.tauri_genome.fna
annots:
  O.tauri_annotation.gff
dataDir:
  Data/
channels:
  - conda-forge
  - bioconda
  - main
  - r
  - default
dependencies:
  # Specify python version (not required but can help with downstream conflicts)
  - python=3.7.6
  # The workflow manager
  - snakemake-minimal=5.10.0
  # For visualizing workflows
  - graphviz=2.42.3
  - xorg-libxrender
  - xorg-libxpm
  # For downloading files
  - wget=1.20.1
  # For the RNAseq analysis
  - fastqc=0.11.9
  - bowtie2=2.4.1
  - samtools=1.10
  - subread=2.0.1
```


**Slurm file - Téléchargement**

In [ ]:
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/runAnalysis.slurm

**Slurm file - Contenu**

``` bash
#!/bin/bash
#
#SBATCH --mem 50GB
#SBATCH --cpus-per-task 30
#SBATCH --partition fast

module load snakemake fastqc bowtie2 samtools subread slurm-drmaa

snakemake --drmaa --jobs=$SLURM_CPUS_PER_TASK -s demo.smk --configfile config.yml

singularity exec -B $PWD:/home/ journee_axe_bioinfo_latest.sif Rscript --vanilla sartools.R
```

**Fichers pour l'analyse différentielle - Téléchargement**

In [ ]:
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/condition.txt
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/sartools.R

**Contenu du script R** 

``` R
library(SARTools)

################################################################################
### R script to compare several conditions with the SARTools and DESeq2 packages
### Hugo Varet
### November 28th, 2019
### designed to be executed with SARTools 1.7.3
################################################################################

################################################################################
###                parameters: to be modified by the user                    ###
################################################################################
rm(list=ls())                                        # remove all the objects from the R session

workDir <- "/home/"      # working directory for the R session

projectName <- "Demo"                         # name of the project
author <- "Thomas Denecker"                                # author of the statistical analysis/report

targetFile <- "condition.txt"                           # path to the design/target file
rawDir <- "Result/Counts"                                      # path to the directory containing raw counts files
featuresToRemove <- c("alignment_not_unique",        # names of the features to be removed
                      "ambiguous", "no_feature",     # (specific HTSeq-count information and rRNA for example)
                      "not_aligned", "too_low_aQual")# NULL if no feature to remove

varInt <- "Iron"                                    # factor of interest
condRef <- "STANDARD"                                      # reference biological condition
batch <- NULL                                        # blocking factor: NULL (default) or "batch" for example

fitType <- "parametric"                              # mean-variance relationship: "parametric" (default), "local" or "mean"
cooksCutoff <- TRUE                                  # TRUE/FALSE to perform the outliers detection (default is TRUE)
independentFiltering <- TRUE                         # TRUE/FALSE to perform independent filtering (default is TRUE)
alpha <- 0.05                                        # threshold of statistical significance
pAdjustMethod <- "BH"                                # p-value adjustment method: "BH" (default) or "BY"

typeTrans <- "VST"                                   # transformation for PCA/clustering: "VST" or "rlog"
locfunc <- "median"                                  # "median" (default) or "shorth" to estimate the size factors

colors <- c("#f3c300", "#875692", "#f38400",         # vector of colors of each biological condition on the plots
            "#a1caf1", "#be0032", "#c2b280",
            "#848482", "#008856", "#e68fac",
            "#0067a5", "#f99379", "#604e97")

forceCairoGraph <- FALSE

################################################################################
###                             running script                               ###
################################################################################
setwd(workDir)
library(SARTools)
if (forceCairoGraph) options(bitmapType="cairo")

# checking parameters
checkParameters.DESeq2(projectName=projectName,author=author,targetFile=targetFile,
                       rawDir=rawDir,featuresToRemove=featuresToRemove,varInt=varInt,
                       condRef=condRef,batch=batch,fitType=fitType,cooksCutoff=cooksCutoff,
                       independentFiltering=independentFiltering,alpha=alpha,pAdjustMethod=pAdjustMethod,
                       typeTrans=typeTrans,locfunc=locfunc,colors=colors)

# loading target file
target <- loadTargetFile(targetFile=targetFile, varInt=varInt, condRef=condRef, batch=batch)

# loading counts
counts <- loadCountData(target=target, rawDir=rawDir, featuresToRemove=featuresToRemove)

# description plots
majSequences <- descriptionPlots(counts=counts, group=target[,varInt], col=colors)

# analysis with DESeq2
out.DESeq2 <- run.DESeq2(counts=counts, target=target, varInt=varInt, batch=batch,
                         locfunc=locfunc, fitType=fitType, pAdjustMethod=pAdjustMethod,
                         cooksCutoff=cooksCutoff, independentFiltering=independentFiltering, alpha=alpha)

# PCA + clustering
exploreCounts(object=out.DESeq2$dds, group=target[,varInt], typeTrans=typeTrans, col=colors)

# summary of the analysis (boxplots, dispersions, diag size factors, export table, nDiffTotal, histograms, MA plot)
summaryResults <- summarizeResults.DESeq2(out.DESeq2, group=target[,varInt], col=colors,
                                          independentFiltering=independentFiltering,
                                          cooksCutoff=cooksCutoff, alpha=alpha)

# save image of the R session
save.image(file=paste0(projectName, ".RData"))

# generating HTML report
writeReport.DESeq2(target=target, counts=counts, out.DESeq2=out.DESeq2, summaryResults=summaryResults,
                   majSequences=majSequences, workDir=workDir, projectName=projectName, author=author,
                   targetFile=targetFile, rawDir=rawDir, featuresToRemove=featuresToRemove, varInt=varInt,
                   condRef=condRef, batch=batch, fitType=fitType, cooksCutoff=cooksCutoff,
                   independentFiltering=independentFiltering, alpha=alpha, pAdjustMethod=pAdjustMethod,
                   typeTrans=typeTrans, locfunc=locfunc, colors=colors)
```

**Contenu du fichier contition.txt**

```
label	files	Iron
D1	SRR3105699_chr18_ftc.txt	DEPLETED
D2	SRR3105698_chr18_ftc.txt	DEPLETED
D3	SRR3105697_chr18_ftc.txt	DEPLETED
S1	SRR3099587_chr18_ftc.txt	STANDARD
S2	SRR3099586_chr18_ftc.txt	STANDARD
S3	SRR3099585_chr18_ftc.txt	STANDARD

```

### Récupération de l'environnement pour l'analyse différentielle

#### Docker 

> Pourquoi ne pas faire dans conda comme précédement  ? 

Un des défauts de R est qu'il n'est pas possible simplement de récupérer une version d'un package. Une solution est donc de figé R et ses packages lors de l'analyse dans une image docker. 

![Figure 6](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/docker_singularity.png)

> Comment as tu construit l'image docker utilisée dans cette analyse ? 

1) Écrire un Dockerfile

``` bash
FROM rocker/tidyverse
MAINTAINER Thomas DENECKER (thomas.denecker@france-bioinformatique.fr)
RUN Rscript -e 'devtools::install_github("PF2-pasteur-fr/SARTools", build_opts="--no-resave-data")'
```
2) Construire à l'aide de `docker build` 

``` bash
docker build -t tdenecker/journee_axe_bioinfo .
```

> Comment as tu partagé cette image ?

``` bash
docker push tdenecker/journee_axe_bioinfo
```

#### Utiliser une image docker sur le cluster via Singularity

Notamment pour des questions de sécurité, il n'est pas possible d'utiliser directement docker sur le cluster de l'IFB mais singularity. Heureusement, il est très simple de construire une image singularity à partir de l'image docker.

In [ ]:
SINGULARITY_LOCALCACHEDIR=${PWD}
SINGULARITY_PULLFOLDER=${PWD}
singularity pull docker://tdenecker/journee_axe_bioinfo

### Étape 5 - Lancement de l'analyse

![Figure 6](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_5.png)

Pour lancer l'analyse, un script bash a été écrit. Il se décompose en 4 grandes parties :
- Les paramètres de SLURM (l'orchestrateur de jobs disponible sur le cluster de l'IFB)
- Le chargement des outils nécessaires à l'aide de `module load`
- Le lancement du workflow Snakemake qui va réaliser toutes les étapes du contrôle qualité des fichiers fastq jusqu'à la génération d'une table de comptage. 
- Le lancement dans l'image singularity de l'analyse différentielle réalisé par SARtools (

In [ ]:
cat runAnalysis.slurm

![Figure 7](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_6.png)
![Figure 8](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_7.png)

In [ ]:
sbatch runAnalysis.slurm

In [ ]:
mv slurm-* Logs/

## Quelques résultats

### Volcano plot

![volcano](demo/figures/volcanoPlot.png)

**Les gènes UP régulés**

In [ ]:
cut -f1,18,20,21 tables/DEPLETEDvsSTANDARD.up.txt 

**Les gènes DOWN régulés**

In [ ]:
cut -f1,18,20,21 tables/DEPLETEDvsSTANDARD.down.txt 

### Export
![Figure 9](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020/raw/main/img/JourneeAxeStep_8.png)

### Exploration avec un Notebook R interactif 

Une petite démo rapide ? 

## Merci !

### La team FAIR Bioinfo 
<div id="banner">
    <div style="display: inline-block;">
        <img src="https://avatars2.githubusercontent.com/u/11072022?s=400&u=49843fd5dfd68775dbdd2c1e8f917a2b45e59436&v=4" alt="Gildas" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
         <img src="https://pbs.twimg.com/profile_images/1141071921007747072/tO6YzJ0A_400x400.jpg" alt="Julien" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://migale.inrae.fr/sites/default/files/helene_chiapello.png" alt="Hélène" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://www.lri.fr///images/membre/430.jpg" alt="Claire" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://www.iscb.org/images/stories/ismbeccb2021/people/jacques.jpg" alt="Jacques" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://pbs.twimg.com/profile_images/486507449856565248/vEeEcItU_400x400.jpeg" alt="Céline" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://media-exp1.licdn.com/dms/image/C4D03AQFPPt_IGy9TGQ/profile-displayphoto-shrink_100_100/0?e=2159024400&v=beta&t=K2l9QULm8YeCVnEcai-oK47GJbVuEKe-xSdNJZI9xIU" alt="Jacques" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://www.cesgo.org/collaboration/wp-content/uploads/sites/2/avatars/200/5d9dfd4bc48e7-bpfull.jpg" alt="Jacques" style="border-radius: 50%;" width="100px">
    </div>
    
</div>

### Et aussi
<div id="banner">
    <div style="display: inline-block;">
       <img src="https://0.academia-photos.com/33133954/9829661/10954491/s200_s..boulakia.jpg" alt="Sarah" style="border-radius: 50%;" width="100px">
    </div>
    <div style="display: inline-block;">
       <img src="https://pbs.twimg.com/profile_images/1294370956094341121/b0UJnjov_400x400.jpg" alt="Gaëlle" style="border-radius: 50%;" width="100px">
    </div>
</div>

## Ressources 

**Cette présentation**
- [GitHub](https://github.com/IFB-ElixirFr/journeeAxeBioinfo_2020) 
- [DockerHub](https://hub.docker.com/repository/docker/tdenecker/journee_axe_bioinfo) 

**FAIR_bioinfo**
- [2019](https://github.com/thomasdenecker/FAIR_Bioinfo) 
- [2020](https://ifb-elixirfr.github.io/IFB-FAIR-bioinfo-training/)

**FAIR & le cluster de l'IFB**
- [Slurm](https://ifb-elixirfr.github.io/IFB-FAIR-bioinfo-training/assets/pdf/Session2020/04_cluster.pdf)
- [Snakemake + Slurm](https://ifb-elixirfr.github.io/IFB-FAIR-bioinfo-training/assets/pdf/Session2020/04_tp1_snakemake.pdf)
- [Docker/Singularity](https://ifb-elixirfr.github.io/IFB-FAIR-bioinfo-training/assets/pdf/Session2020/04_tp2_singularity.pdf)

**Données**
- [Originale](https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR4026187)
- Réduite : [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3997237.svg)](https://doi.org/10.5281/zenodo.3997237)

**Publication**
- Lelandais, G., Scheiber, I., Paz-Yepes, J. et al. Ostreococcus tauri is a new model green alga for studying iron metabolism in eukaryotic phytoplankton. BMC Genomics 17, 319 (2016). https://doi.org/10.1186/s12864-016-2666-6

**Outils utilisés**
- Snakemake [![Snakemake](https://img.shields.io/badge/snakemake-≥5.6.0-brightgreen.svg?style=flat)](https://snakemake.readthedocs.io)
- [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)
- Bowtie2 : Langmead B, Salzberg S. Fast gapped-read alignment with Bowtie 2. Nature Methods. 2012, 9:357-359.
- SAMtools : Li H., Handsaker B., Wysoker A., Fennell T., Ruan J., Homer N., Marth G., Abecasis G., Durbin R. and 1000 Genome Project Data Processing Subgroup (2009) The Sequence alignment/map (SAM) format and SAMtools. Bioinformatics, 25, 2078-9. [PMID: 19505943](http://www.ncbi.nlm.nih.gov/pubmed/19505943)
- featureCounts : Liao Y, Smyth GK, Shi W. featureCounts: an efficient general purpose program for assigning sequence reads to genomic features. Bioinformatics. 2014 Apr 1;30(7):923-30. doi: 10.1093/bioinformatics/btt656. Epub 2013 Nov 13. PMID: 24227677.
- SARtools : H. Varet, L. Brillet-Guéguen, J.-Y. Coppee and M.-A. Dillies, SARTools: A DESeq2- and EdgeR-Based R Pipeline for Comprehensive Differential Analysis of RNA-Seq Data, PLoS One, 2016, doi: http://dx.doi.org/10.1371/journal.pone.0157022 
- R : R Core Team (2020). R: A language and environment for statistical computing. R Foundation for Statistical Computing, Vienna, Austria.URL https://www.R-project.org/.